## XSBench
Compiler: `gcc-7.2.0`  
flags: `-std=gnu99 -march=native -fopenmp -g -pg -O3`  
libs: `-lm`   
  
(Haswell) Intel(R) Xeon(R) CPU E5-2699 v3 @ 2.30GHz  
`divsd`: 10-20 Cycles  
`movsd`: 3 cycles  
L1 Cache: 32 kB, 8 way, 64 sets, 64 B line size, **latency 4**, per core.    
L2 Cache: 256 kB, 8 way, 512 sets, 64 B line size, **latency 12**, per core.  
L3 Cache: 45 MB, 12-16 way, 64 B line size, **latency 34**, shared.

|   | L1 Cache | L2 Cache | L3 Cache | DRAM |
|:---|:------:|:--------:|:----------:|:-----|
|**Cache Lines / Cycle** | 1.9728 | 1.2120 | 0.7745 | 0.4402 |


### Serial Run
---
####  `CalculateXS.c::calculate_micro_xs( ) | Line 41`: 
|  CPUTIME  | Instructions per Cycle | L1 Data Cache Miss Rate | L2 Data Cache Miss Rate |
|:---------:|:----------------------:|:-----------------------:|:-----------------------:|
| 7.7 %     |    **0.25**            | 13.9%                   |     88.0%               | 
```c
|40|	// calculate the re-useable interpolation factor
|41|	f = (high->energy - p_energy) / (high->energy - low->energy); 
```

---
#### `XSutils.c::binary_search( ) | Loop at 59`:

|  CPUTIME  | Instructions per Cycle | L1 Data Cache Miss Rate | L2 Data Cache Miss Rate |
|:---------:|:----------------------:|:-----------------------:|:-----------------------:|
| 78.9%     |    0.96                | **29.4%**               |     87.9%               |  
```c
	// Begins binary search	 // Conditional Branch Misprediction Rate | CPUTIME
|59|	while( max >= min )                       //              .5% | 78.9%
|60|	{
|61|		mid = min + floor( (max-min) / 2.0);  //                  | 17.8% 
|62|		if( A[mid].energy < quarry )          //             8.3% | 55.8%
|63|			min = mid+1;
|64|        else if( A[mid].energy > quarry )     //                  |  2.3%
|65|			max = mid-1;
|66|		else
|67|			return mid;
|68|    }
```


### 72 Thread Run
---
####  `CalculateXS.c::calculate_micro_xs( )`: 
|  CPUTIME  | Instructions per Cycle | L1 Data Cache Miss Rate | L2 Data Cache Miss Rate |
|:---------:|:----------------------:|:-----------------------:|:-----------------------:|
| 3.8 %     |    **0.15**            | 8.0%                   |     74.8%               | 
```c
|40|	// calculate the re-useable interpolation factor
|41|	f = (high->energy - p_energy) / (high->energy - low->energy); 
```

---
#### `XSutils.c::binary_search( )`:

|  CPUTIME  | Instructions per Cycle | L1 Data Cache Miss Rate | L2 Data Cache Miss Rate |
|:---------:|:----------------------:|:-----------------------:|:-----------------------:|
| 20.9%     |    0.37               | **77.7%**               |     93.2%               |  
```c
|44|// Binary Search function for nuclide grid
|45|// Returns ptr to energy less than the quarry that is closest to the quarry
|46|int binary_search( NuclideGridPoint * A, double quarry, int n )
|47|{       
|48|	int min = 0;
|49|	int max = n-1;
|50|	int mid;
|51|	
|52|	// checks to ensure we're not reading off the end of the grid
|53|	if( A[0].energy > quarry )           // 4.4% Branch Mispredict Rate
|54|		return 0;
|55|	else if( A[n-1].energy < quarry )
|56|		return n-2;
|57|	
|58|	// Begins binary search	
|59|	while( max >= min )
|60|	{    
|61|        	mid = min + floor( (max-min) / 2.0);
|62|		if( A[mid].energy < quarry )     // 85.7% Branch Mispredict Rate
|63|			min = mid+1;
|64|		else if( A[mid].energy > quarry )
|65|			max = mid-1;
|66|		else
|67|			return mid;
|68|	}
|69|       	return max;
|70|}
```



`__kmp_wait_yield_4` : 54% CPUTIME

Pushes Bottle Neck to backend with increase in Microcode Sequencer Instructions and Cache Misses Latency

In [15]:

2.1e+09 / 2.44e+09

0.860655737704918